Includes:

Gettign the NDVI dataset

Filtering it on the year and coordinates to match the rest of the data

Binning into distinct 

Aggregating to per season statistics

### Libarires

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rasterio
import rasterio.features

In [3]:
import pandas as pd

In [4]:
import rioxarray
import geopandas as gpd
from rasterstats import zonal_stats

### Loading data

In [6]:
gdf = gpd.read_file("C:/Users/Dima/Desktop/MIP/stations/regions_selected_labeled.gpkg")

In [7]:
gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom.buffer(0) if not geom.is_valid else geom)

In [8]:
gdf.geometry = gdf.geometry.to_crs("EPSG:4326")

In [9]:
ndvi_frame = xr.open_dataset("AusENDVI-noclim_MCD43A4_1982_2022_0.1.0.nc")
ndvi_frame

<xarray.Dataset> Size: 1GB
Dimensions:                  (time: 492, latitude: 680, longitude: 840)
Coordinates:
  * longitude                (longitude) float32 3kB 112.0 112.1 ... 153.9 154.0
  * latitude                 (latitude) float32 3kB -10.02 -10.07 ... -43.97
    spatial_ref              int32 4B ...
  * time                     (time) datetime64[ns] 4kB 1982-01-15 ... 2022-12-15
Data variables:
    AusENDVI_noclim_MCD43A4  (time, latitude, longitude) float32 1GB ...
Attributes:
    nodata:       nan
    crs:          EPSG:4326
    description:  This dataset consists of calibrated and harmonised (to MODI...
    units:        unitless
    long_name:    AusENDVI_noclim merged with MODIS-MCD43A4 NDVI

In [10]:
times = ndvi_frame.time.to_index()
times

DatetimeIndex(['1982-01-15', '1982-02-15', '1982-03-15', '1982-04-15',
               '1982-05-15', '1982-06-15', '1982-07-15', '1982-08-15',
               '1982-09-15', '1982-10-15',
               ...
               '2022-03-15', '2022-04-15', '2022-05-15', '2022-06-15',
               '2022-07-15', '2022-08-15', '2022-09-15', '2022-10-15',
               '2022-11-15', '2022-12-15'],
              dtype='datetime64[ns]', name='time', length=492, freq=None)

In [11]:
start_date = "2006-01-01"
end_date = "2020-12-31"
years_mask = (times >= start_date) & (times <= end_date)

In [12]:
ndvi_subframe = ndvi_frame.sel(time = years_mask)

In [13]:
months_mask = ndvi_subframe.time.dt.month.isin([9,10,11,12,1,2,3,4,5])

In [14]:
ndvi_subframe = ndvi_subframe.sel(time = months_mask)

In [15]:
ndvi_subframe

<xarray.Dataset> Size: 308MB
Dimensions:                  (time: 135, latitude: 680, longitude: 840)
Coordinates:
  * longitude                (longitude) float32 3kB 112.0 112.1 ... 153.9 154.0
  * latitude                 (latitude) float32 3kB -10.02 -10.07 ... -43.97
    spatial_ref              int32 4B ...
  * time                     (time) datetime64[ns] 1kB 2006-01-15 ... 2020-12-15
Data variables:
    AusENDVI_noclim_MCD43A4  (time, latitude, longitude) float32 308MB ...
Attributes:
    nodata:       nan
    crs:          EPSG:4326
    description:  This dataset consists of calibrated and harmonised (to MODI...
    units:        unitless
    long_name:    AusENDVI_noclim merged with MODIS-MCD43A4 NDVI

In [16]:
print("Min: " , ndvi_subframe["AusENDVI_noclim_MCD43A4"].min().item())
print("Max: " , ndvi_subframe["AusENDVI_noclim_MCD43A4"].max().item())

Min:  -1.0
Max:  0.943803608417511


In [17]:
bins = [-1.0, 0, 0.1, 0.3, 0.5, 0.7, 1.0]

In [18]:
bin_labels = [
    "Water/Snow/Clouds", "Bare Soil", "Sparse Veg", 
    "Moderate Veg", "Dense Veg", "Very Dense Veg"
]

In [19]:
def get_binned_hist_from_array(geom, ndvi_array, transform, bins):
    """
    Compute binned histogram from a 2D numpy NDVI array and region geometry
    """
    # Generate a mask
    mask = rasterio.features.geometry_mask(
        [geom], transform=transform, invert=True, out_shape=ndvi_array.shape
    )

    # Get masked values
    values = ndvi_array[mask]

    # Remove masked / invalide values
    values = values[~np.isnan(values)]

    # Get histogram
    counts, _ = np.histogram(values, bins=bins)

    total = counts.sum()
    if total == 0:
        return [0] * len(counts)
    return (counts / total).tolist()

In [20]:
def extract_ndvi_bins_timeseries(ds, gdf, bins, bin_labels):
    """
    Extract binned NDVI histogram per region and per month as a DataFrame.
    """
    transform = ds.rio.transform()
    records = []

    for time in ds.time:
        ndvi_slice = ds.sel(time=time).values
        dt = pd.to_datetime(time.values)
        year = dt.year
        month = dt.month
        day = dt.day
        date = dt

        for i, geom in enumerate(gdf.geometry):
            region_name = gdf.iloc[i]['RegionName']
            bin_values = get_binned_hist_from_array(geom, ndvi_slice, transform, bins)
            
            record = {
                'RegionName': region_name,
                'Date' : date,
                'Year': year,
                'Month': month,
                'Day' : day
            }
            
            # Add bin percentages
            for label, value in zip(bin_labels, bin_values):
                record[label] = value
            
            records.append(record)
    
    return pd.DataFrame(records)

In [21]:
df_bins = extract_ndvi_bins_timeseries(ndvi_subframe["AusENDVI_noclim_MCD43A4"], gdf, bins, bin_labels)

In [22]:
df_bins.head()

,RegionName,Date,Year,Month,Day,Water/Snow/Clouds,Bare Soil,Sparse Veg,Moderate Veg,Dense Veg,Very Dense Veg
0,QLD Condamine-Balonne,2006-01-15,2006,1,15,0.000000,0.000000,0.609082,0.295880,0.089419,0.005618
1,NSW Barwon Darling,2006-01-15,2006,1,15,0.000000,0.000000,0.900919,0.096241,0.002840,0.000000
2,QLD Moonie,2006-01-15,2006,1,15,0.000000,0.000000,0.720358,0.275168,0.004474,0.000000
3,NSW Murray Above,2006-01-15,2006,1,15,0.000000,0.000000,0.000000,0.452703,0.466216,0.081081
4,NSW Lower Darling,2006-01-15,2006,1,15,0.001614,0.000646,0.941879,0.055861,0.000000,0.000000


In [23]:
df_bins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RegionName         1620 non-null   object        
 1   Date               1620 non-null   datetime64[ns]
 2   Year               1620 non-null   int64         
 3   Month              1620 non-null   int64         
 4   Day                1620 non-null   int64         
 5   Water/Snow/Clouds  1620 non-null   float64       
 6   Bare Soil          1620 non-null   float64       
 7   Sparse Veg         1620 non-null   float64       
 8   Moderate Veg       1620 non-null   float64       
 9   Dense Veg          1620 non-null   float64       
 10  Very Dense Veg     1620 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(3), object(1)
memory usage: 139.3+ KB


In [24]:
df_bins.to_csv("ndvi_binned.csv", index = False)

### mapping stages

In [24]:
def map_north(dates):
    if ((dates.Month == 9) and (dates.Day > 20)) or (dates.Month == 10):
        return "Planting"
    elif (dates.Month == 11) or (dates.Month == 12):
        return "Growing"
    elif (dates.Month == 1):
        return "Flowering"
    elif (dates.Month == 2) or (dates.Month == 3):
        return "Maturation"
    elif (dates.Month == 4):
        return "Harvesting"
    else:
        return "NotSeason"

In [25]:
def map_south(dates):
    if ((dates.Month == 10) and (dates.Day >= 15)) or (dates.Month == 11):
        return "Planting"
    elif (dates.Month == 12):
        return "Growing"
    elif (dates.Month == 1):
        return "Flowering"
    elif (dates.Month == 2) or (dates.Month == 3):
        return "Maturation"
    elif (dates.Month == 4) or (dates.Month == 5):
        return "Harvesting"
    else:
        return "NotSeason"

In [26]:
def season_mapping(frame):
    north = ["QLD Condamine-Balonne", "QLD Moonie" ,"QLD Border Rivers", "NSW Border Rivers", "NSW Barwon-Darling", "NSW Lower Darling", "NSW Macquarie-Castlereagh", "NSW Namoi", "NSW Gwydir"]
    # south = ["NSW Murrumbidgee", "NSW Lachlan", "NSW Murray Above"]

    if frame.RegionName in north:
        return map_north(frame)
    else:
        return map_south(frame)

In [27]:
df_bins["Season"] = df_bins[["RegionName", "Date", "Month", "Day"]].apply(season_mapping, axis = 1)
df_bins

,RegionName,Date,Year,Month,Day,Water/Snow/Clouds,Bare Soil,Sparse Veg,Moderate Veg,Dense Veg,Very Dense Veg,Season
0,QLD Condamine-Balonne,2006-01-15,2006,1,15,0.000000,0.000000,0.609082,0.295880,0.089419,0.005618,Flowering
1,NSW Barwon Darling,2006-01-15,2006,1,15,0.000000,0.000000,0.900919,0.096241,0.002840,0.000000,Flowering
2,QLD Moonie,2006-01-15,2006,1,15,0.000000,0.000000,0.720358,0.275168,0.004474,0.000000,Flowering
3,NSW Murray Above,2006-01-15,2006,1,15,0.000000,0.000000,0.000000,0.452703,0.466216,0.081081,Flowering
4,NSW Lower Darling,2006-01-15,2006,1,15,0.001614,0.000646,0.941879,0.055861,0.000000,0.000000,Flowering
...,...,...,...,...,...,...,...,...,...,...,...,...
1615,NSW Gwydir,2020-12-15,2020,12,15,0.000000,0.000000,0.443023,0.305814,0.251163,0.000000,Growing
1616,NSW Namoi,2020-12-15,2020,12,15,0.000000,0.000000,0.250000,0.528631,0.221369,0.000000,Growing
1617,NSW Macquarie-Castlereagh,2020-12-15,2020,12,15,0.000000,0.000000,0.399707,0.414008,0.182985,0.003300,Growing
1618,NSW Lachlan,2020-12-15,2020,12,15,0.000000,0.000612,0.497706,0.397675,0.102172,0.001835,Growing


In [28]:
df_bins = df_bins[df_bins["Season"] != "NotSeason"]
df_bins

,RegionName,Date,Year,Month,Day,Water/Snow/Clouds,Bare Soil,Sparse Veg,Moderate Veg,Dense Veg,Very Dense Veg,Season
0,QLD Condamine-Balonne,2006-01-15,2006,1,15,0.000000,0.000000,0.609082,0.295880,0.089419,0.005618,Flowering
1,NSW Barwon Darling,2006-01-15,2006,1,15,0.000000,0.000000,0.900919,0.096241,0.002840,0.000000,Flowering
2,QLD Moonie,2006-01-15,2006,1,15,0.000000,0.000000,0.720358,0.275168,0.004474,0.000000,Flowering
3,NSW Murray Above,2006-01-15,2006,1,15,0.000000,0.000000,0.000000,0.452703,0.466216,0.081081,Flowering
4,NSW Lower Darling,2006-01-15,2006,1,15,0.001614,0.000646,0.941879,0.055861,0.000000,0.000000,Flowering
...,...,...,...,...,...,...,...,...,...,...,...,...
1615,NSW Gwydir,2020-12-15,2020,12,15,0.000000,0.000000,0.443023,0.305814,0.251163,0.000000,Growing
1616,NSW Namoi,2020-12-15,2020,12,15,0.000000,0.000000,0.250000,0.528631,0.221369,0.000000,Growing
1617,NSW Macquarie-Castlereagh,2020-12-15,2020,12,15,0.000000,0.000000,0.399707,0.414008,0.182985,0.003300,Growing
1618,NSW Lachlan,2020-12-15,2020,12,15,0.000000,0.000612,0.497706,0.397675,0.102172,0.001835,Growing


In [29]:
pivoted_water = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Water/Snow/Clouds"
).add_prefix("water" + "_").reset_index()
pivoted_water

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
pivoted_bare = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Bare Soil"
).add_prefix("bare" + "_").reset_index()
pivoted_bare

Season,RegionName,Year,bare_Flowering,bare_Growing,bare_Harvesting,bare_Maturation,bare_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000835,0.000000,0.000000,0.000251
2,NSW Barwon Darling,2008,0.000835,0.001170,0.001170,0.002339,0.000835
3,NSW Barwon Darling,2009,0.000835,0.000835,0.000418,0.000585,0.000752
4,NSW Barwon Darling,2010,0.000835,0.001170,0.002089,0.002005,0.001671
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
pivoted_sparse = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Sparse Veg"
).add_prefix("sparse" + "_").reset_index()
pivoted_sparse

Season,RegionName,Year,sparse_Flowering,sparse_Growing,sparse_Harvesting,sparse_Maturation,sparse_Planting
0,NSW Barwon Darling,2006,0.900919,0.962406,0.757644,0.870927,0.915121
1,NSW Barwon Darling,2007,0.946700,0.879699,0.778028,0.925313,0.964495
2,NSW Barwon Darling,2008,0.704428,0.865163,0.608521,0.557226,0.822640
3,NSW Barwon Darling,2009,0.921303,0.956892,0.645196,0.786550,0.930744
4,NSW Barwon Darling,2010,0.655639,0.238764,0.299248,0.391479,0.298997
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.398210,0.636465,0.630872,0.587248,0.004474
176,QLD Moonie,2017,0.765101,0.323266,0.277405,0.598434,0.214765
177,QLD Moonie,2018,0.787472,0.724832,0.456376,0.408277,0.447427
178,QLD Moonie,2019,0.876957,0.929530,0.610738,0.895973,0.919463


In [32]:
pivoted_moderate = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Moderate Veg"
).add_prefix("moderate" + "_").reset_index()
pivoted_moderate

Season,RegionName,Year,moderate_Flowering,moderate_Growing,moderate_Harvesting,moderate_Maturation,moderate_Planting
0,NSW Barwon Darling,2006,0.096241,0.037594,0.238513,0.125230,0.084294
1,NSW Barwon Darling,2007,0.053300,0.117794,0.219465,0.073851,0.035171
2,NSW Barwon Darling,2008,0.283876,0.130493,0.384127,0.405013,0.167251
3,NSW Barwon Darling,2009,0.076692,0.042272,0.348371,0.209273,0.068254
4,NSW Barwon Darling,2010,0.335338,0.721470,0.658396,0.534169,0.643943
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.597315,0.357942,0.337808,0.403803,0.861298
176,QLD Moonie,2017,0.230425,0.649888,0.456376,0.369128,0.760626
177,QLD Moonie,2018,0.208054,0.272931,0.494407,0.497763,0.534676
178,QLD Moonie,2019,0.120805,0.070470,0.351230,0.098434,0.080537


In [33]:
pivoted_dense = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Dense Veg"
).add_prefix("dense" + "_").reset_index()
pivoted_dense

Season,RegionName,Year,dense_Flowering,dense_Growing,dense_Harvesting,dense_Maturation,dense_Planting
0,NSW Barwon Darling,2006,0.002840,0.000000,0.003843,0.003843,0.000585
1,NSW Barwon Darling,2007,0.000000,0.001671,0.002506,0.000835,0.000084
2,NSW Barwon Darling,2008,0.009858,0.002840,0.005681,0.034252,0.008354
3,NSW Barwon Darling,2009,0.000835,0.000000,0.005597,0.003258,0.000084
4,NSW Barwon Darling,2010,0.007853,0.037427,0.038262,0.070927,0.053133
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.004474,0.005593,0.031320,0.008949,0.129754
176,QLD Moonie,2017,0.004474,0.026846,0.266219,0.032438,0.024609
177,QLD Moonie,2018,0.004474,0.002237,0.049217,0.093960,0.017897
178,QLD Moonie,2019,0.002237,0.000000,0.038031,0.005593,0.000000


In [34]:
pivoted_very_dense = df_bins.pivot_table(
    index = ["RegionName", "Year"], 
    columns = "Season", 
    values = "Very Dense Veg"
).add_prefix("very_dense" + "_").reset_index()
pivoted_very_dense

Season,RegionName,Year,very_dense_Flowering,very_dense_Growing,very_dense_Harvesting,very_dense_Maturation,very_dense_Planting
0,NSW Barwon Darling,2006,0.0,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.0,0.000000,0.000000,0.000000,0.000000
2,NSW Barwon Darling,2008,0.0,0.000000,0.000000,0.000000,0.000000
3,NSW Barwon Darling,2009,0.0,0.000000,0.000000,0.000000,0.000000
4,NSW Barwon Darling,2010,0.0,0.000167,0.000501,0.000919,0.001253
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.0,0.000000,0.000000,0.000000,0.004474
176,QLD Moonie,2017,0.0,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.0,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.0,0.000000,0.000000,0.000000,0.000000


### Combining

In [35]:
ndvi_combined = pivoted_water.copy()
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003
...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
ndvi_combined = ndvi_combined.merge(pivoted_bare, on = ["RegionName", "Year"], how = "inner")
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,bare_Maturation,bare_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,0.000000,0.000251
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,0.002339,0.000835
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,0.000585,0.000752
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,0.002005,0.001671
...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [37]:
ndvi_combined = ndvi_combined.merge(pivoted_sparse, on = ["RegionName", "Year"], how = "inner")
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,bare_Maturation,bare_Planting,sparse_Flowering,sparse_Growing,sparse_Harvesting,sparse_Maturation,sparse_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.900919,0.962406,0.757644,0.870927,0.915121
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,0.000000,0.000251,0.946700,0.879699,0.778028,0.925313,0.964495
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,0.002339,0.000835,0.704428,0.865163,0.608521,0.557226,0.822640
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,0.000585,0.000752,0.921303,0.956892,0.645196,0.786550,0.930744
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,0.002005,0.001671,0.655639,0.238764,0.299248,0.391479,0.298997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.398210,0.636465,0.630872,0.587248,0.004474
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.765101,0.323266,0.277405,0.598434,0.214765
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.787472,0.724832,0.456376,0.408277,0.447427
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.876957,0.929530,0.610738,0.895973,0.919463


In [38]:
ndvi_combined = ndvi_combined.merge(pivoted_moderate, on = ["RegionName", "Year"], how = "inner")
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,...,sparse_Flowering,sparse_Growing,sparse_Harvesting,sparse_Maturation,sparse_Planting,moderate_Flowering,moderate_Growing,moderate_Harvesting,moderate_Maturation,moderate_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.900919,0.962406,0.757644,0.870927,0.915121,0.096241,0.037594,0.238513,0.125230,0.084294
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,...,0.946700,0.879699,0.778028,0.925313,0.964495,0.053300,0.117794,0.219465,0.073851,0.035171
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,...,0.704428,0.865163,0.608521,0.557226,0.822640,0.283876,0.130493,0.384127,0.405013,0.167251
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,...,0.921303,0.956892,0.645196,0.786550,0.930744,0.076692,0.042272,0.348371,0.209273,0.068254
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,...,0.655639,0.238764,0.299248,0.391479,0.298997,0.335338,0.721470,0.658396,0.534169,0.643943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.398210,0.636465,0.630872,0.587248,0.004474,0.597315,0.357942,0.337808,0.403803,0.861298
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.765101,0.323266,0.277405,0.598434,0.214765,0.230425,0.649888,0.456376,0.369128,0.760626
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.787472,0.724832,0.456376,0.408277,0.447427,0.208054,0.272931,0.494407,0.497763,0.534676
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.876957,0.929530,0.610738,0.895973,0.919463,0.120805,0.070470,0.351230,0.098434,0.080537


In [39]:
ndvi_combined =  ndvi_combined.merge(pivoted_dense, on = ["RegionName", "Year"], how = "inner")
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,...,moderate_Flowering,moderate_Growing,moderate_Harvesting,moderate_Maturation,moderate_Planting,dense_Flowering,dense_Growing,dense_Harvesting,dense_Maturation,dense_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.096241,0.037594,0.238513,0.125230,0.084294,0.002840,0.000000,0.003843,0.003843,0.000585
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,...,0.053300,0.117794,0.219465,0.073851,0.035171,0.000000,0.001671,0.002506,0.000835,0.000084
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,...,0.283876,0.130493,0.384127,0.405013,0.167251,0.009858,0.002840,0.005681,0.034252,0.008354
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,...,0.076692,0.042272,0.348371,0.209273,0.068254,0.000835,0.000000,0.005597,0.003258,0.000084
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,...,0.335338,0.721470,0.658396,0.534169,0.643943,0.007853,0.037427,0.038262,0.070927,0.053133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.597315,0.357942,0.337808,0.403803,0.861298,0.004474,0.005593,0.031320,0.008949,0.129754
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.230425,0.649888,0.456376,0.369128,0.760626,0.004474,0.026846,0.266219,0.032438,0.024609
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.208054,0.272931,0.494407,0.497763,0.534676,0.004474,0.002237,0.049217,0.093960,0.017897
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.120805,0.070470,0.351230,0.098434,0.080537,0.002237,0.000000,0.038031,0.005593,0.000000


In [40]:
ndvi_combined =  ndvi_combined.merge(pivoted_very_dense, on = ["RegionName", "Year"], how = "inner")
ndvi_combined

Season,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,...,dense_Flowering,dense_Growing,dense_Harvesting,dense_Maturation,dense_Planting,very_dense_Flowering,very_dense_Growing,very_dense_Harvesting,very_dense_Maturation,very_dense_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002840,0.000000,0.003843,0.003843,0.000585,0.0,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,...,0.000000,0.001671,0.002506,0.000835,0.000084,0.0,0.000000,0.000000,0.000000,0.000000
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,...,0.009858,0.002840,0.005681,0.034252,0.008354,0.0,0.000000,0.000000,0.000000,0.000000
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,...,0.000835,0.000000,0.005597,0.003258,0.000084,0.0,0.000000,0.000000,0.000000,0.000000
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,...,0.007853,0.037427,0.038262,0.070927,0.053133,0.0,0.000167,0.000501,0.000919,0.001253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.005593,0.031320,0.008949,0.129754,0.0,0.000000,0.000000,0.000000,0.004474
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.026846,0.266219,0.032438,0.024609,0.0,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.002237,0.049217,0.093960,0.017897,0.0,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002237,0.000000,0.038031,0.005593,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [42]:
ndvi_combined.to_csv("ndvi_seasons.csv", index = False)

In [41]:
check = pd.read_csv("ndvi_seasons.csv")
check

,RegionName,Year,water_Flowering,water_Growing,water_Harvesting,water_Maturation,water_Planting,bare_Flowering,bare_Growing,bare_Harvesting,...,dense_Flowering,dense_Growing,dense_Harvesting,dense_Maturation,dense_Planting,very_dense_Flowering,very_dense_Growing,very_dense_Harvesting,very_dense_Maturation,very_dense_Planting
0,NSW Barwon Darling,2006,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002840,0.000000,0.003843,0.003843,0.000585,0.0,0.000000,0.000000,0.000000,0.000000
1,NSW Barwon Darling,2007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000835,0.000000,...,0.000000,0.001671,0.002506,0.000835,0.000084,0.0,0.000000,0.000000,0.000000,0.000000
2,NSW Barwon Darling,2008,0.001003,0.000334,0.000501,0.001170,0.000919,0.000835,0.001170,0.001170,...,0.009858,0.002840,0.005681,0.034252,0.008354,0.0,0.000000,0.000000,0.000000,0.000000
3,NSW Barwon Darling,2009,0.000334,0.000000,0.000418,0.000334,0.000167,0.000835,0.000835,0.000418,...,0.000835,0.000000,0.005597,0.003258,0.000084,0.0,0.000000,0.000000,0.000000,0.000000
4,NSW Barwon Darling,2010,0.000334,0.001003,0.001504,0.000501,0.001003,0.000835,0.001170,0.002089,...,0.007853,0.037427,0.038262,0.070927,0.053133,0.0,0.000167,0.000501,0.000919,0.001253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,QLD Moonie,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.005593,0.031320,0.008949,0.129754,0.0,0.000000,0.000000,0.000000,0.004474
176,QLD Moonie,2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.026846,0.266219,0.032438,0.024609,0.0,0.000000,0.000000,0.000000,0.000000
177,QLD Moonie,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.004474,0.002237,0.049217,0.093960,0.017897,0.0,0.000000,0.000000,0.000000,0.000000
178,QLD Moonie,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002237,0.000000,0.038031,0.005593,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
